<a href="https://colab.research.google.com/github/Keaveney-E/Soundscape_mapper/blob/main/Soundscape_Mapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Outline

The package soundscape_mapper is an open-source Python toolbox that utilizes non-negative matrix factorization (NMF) in audio source separation. This is a quick start guide for the application of soundscape_mapper in acoustic analysis. For more information, please visit our Github.

This guide contains four sections:

Audio visualization
Model training
Deployment and spectrogram reconstruction
Sound source visualisation

Installation
Dependancies:

Python>=3.7

numpy==1.21.6

pandas==1.3.5

audioread==2.1.9

librosa==0.8.1

scikit-learn==1.0.2

scipy==1.7.3

matplotlib==3.2.2

plotly==5.5.0

To install soundscape_mapper, clone the repository in your Python environment.

In [ ]:
# Clone soundscape_mapper from GitHub @Keaveney-e-org
git clone https://github.com/keaveney-e-org/soundscape_mapper.git

Then, install the requirements.txt in the package folder for installing required packages.

In [ ]:
# Install required packages
cd soundscape_mapper
pip install -r requirements.txt

Quick start

Audio Visualisation

soundscape_mapper provides a function audio_visualization to transform an audio into a spectrogram on the hertz or mel scale. It also enables the use of Welch’s averaging method and spectrogram prewhitening in noise reduction. This example uses a short audio clip of dolphin and seal calls to demonstrate the ecoacoustic application of source separation.

In [ ]:
from soundscape_mapper.soundscape_viewer import audio_visualization
# Define spectrogram parameters
sound_train = audio_visualization(filename='copelands_w2019.wav', path='./data/wav/', offset_read=0, duration_read=15,
                                  FFT_size=512, time_resolution=0.1, prewhiten_percent=10, f_range=[0,8000])

2. Model Training

After preparing the training spectrogram, we can train a model with source_separation. NMF learns a set of basis functions to reconstruct the training spectrogram. In soundscape_mapper, we can apply PC-NMF to separate the basis functions into two groups according to their source-specific periodicity. In this example, one group of basis funcitons is associated with dolphin call (mainly < 10 kHz) and another group is associated with noise (mainly > 3.5 kHz). Save the model for further applications.

In [ ]:
from soundscape_mapper.soundscape_viewer import source_separation

# Define model parameters
model=source_separation(feature_length=30, basis_num=10)

# Feature learning
model.learn_feature(input_data=copelands_w2019.data, f=copelands_w2019.f, method='PCNMF')

# Plot the basis functions of two sound source
model.plot_nmf(plot_type='W', source=1)
model.plot_nmf(plot_type='W', source=2)

# Save the model
model.save_model(filename='./data/model/wind_model.mat')

3. Deployment and spectrogram reconstruction

Generate another spectrogram for testing the source separation model.

In [ ]:
# Prepare a spectrogram
sound_predict=audio_visualization(filename='copelands_w2019.wav', path='./data/wav/', offset_read=30, duration_read=15,
                                    FFT_size=512, time_resolution=0.1, prewhiten_percent=10, f_range=[0,8000])

Load the saved model and perform source separation. After the prediction procedure, plot the reconstructed spectrograms to evaluate the separation of deer calls and noise.

In [ ]:
# Deploy the model
model=source_separation()
model.load_model(filename='./data/model/wind_model.mat')
model.prediction(input_data=sound_predict.data, f=sound_predict.f)

# View individual reconstructed spectrogram
model.plot_nmf(plot_type = 'separation', source = 1)
model.plot_nmf(plot_type = 'separation', source = 2)

4. Presence detection

With the reconstructed spectrogram, we can use the function spectrogram_detection to detect the presence of target signals (e.g., dolphin calls, peaks in wind-driven ambient noise ect..). This function will generate a txt file contains the beginning time, ending time, minimum frequency, and maximum frequency of each detected call. Explore the detection result in Raven software.

In [ ]:
from soundscape_mapper.soundscape_viewer import spectrogram_detection

# Choose the source for signal detection
source_num = 1
    
# Define the detection parameters
sp=spectrogram_detection(model.separation[source_num-1], model.f, threshold=5.5, smooth=1, minimum_interval=0.5, 
                           filename='wind_detection.txt', path='./data/txt/')